In [ ]:
import numpy as np
from tools import *
import seaborn as sns; sns.set_theme()
from matplotlib.pyplot import figure
sns.set(rc={'figure.figsize':(18,8)})
%load_ext autoreload
%autoreload 2

In [ ]:
noise = 0.000001
N = 10


Exps = np.sort(np.array([1.5, -0.15, -0.8]))

N_max = 10
Ns = np.arange(3, N_max + 1, 3)
num_runs = 20
T = 2 * N_max
shifts = np.arange(1, 11, 3)
shift_max = max(shifts)

time = np.arange(0, 2.55, 0.1)
Inits = np.array([0.01, 0.2, 1])[::-1] #* (1 + 0 * 0.1 * np.random.randn(3))
Xseries = np.exp(time.reshape(-1, 1) * Exps.reshape(1, -1)) * Inits.reshape(
    1, -1
)

XseriesNoisy = Xseries * (1 + noise * np.random.randn(*Xseries.shape))
XsereisTotal = XseriesNoisy.sum(1)

plt.plot(time, XsereisTotal);
plt.plot(time, Xseries);
plt.show()

proj_series, evec = max_eig_project(XsereisTotal, N, shift=1)

X_target = XseriesNoisy[:, -1]

scalar = 1 / Xseries[N - 1, -1] *  proj_series[0]
scalar = 1/ np.mean(proj_series / X_target[len(X_target)-len(proj_series):])

plt.plot(time, X_target)
plt.plot(time[len(time)-len(proj_series):], proj_series*scalar)
plt.show()

In [ ]:
N = 1000
noise = 0.000001
dim = 50
shift = 10
L  = N-dim-shift
center = False

shifts = [3]
dims = np.linspace(50,500,1, dtype=int)

ys = []
for shift in shifts:
    for dim in dims:
        x = np.linspace(-8,8,N)
        sig1 = tanh(x, a=0.8, lam=1)
        sig2 =  tanh(x, a=0.5, lam=1, center=-4)[::-1]


        sig = sig1 + sig2 + noise * np.random.normal(size=N, scale=1)



        proj_series, evec = max_eig_project(sig, dim, shift=shift, norm=sig2[-1])
        figure(figsize = (4, 5), dpi = 80)
        plt.plot(evec); plt.show()
        
        proj_series = normalize(proj_series)
        plt.plot(proj_series); plt.plot(sig1, '--'); plt.plot(sig2, '--'); plt.plot(sig);
        plt.legend(['Eig Vec', 'Projected', 'C1', 'C2', 'Sum' ])
        plt.title(f'shift={shift}, dim={dim}')
        plt.show()


        y = np.convolve(sig, evec, mode='valid')
        ys.append(y)


In [ ]:

N = 1000
noise = 0.000001
x = np.linspace(-8,8,N)
num_components = 3
sig, basis = build_signal_grid(x, num_components, a=[1,1], lam=[1,1], noise=noise, return_basis=True, center=False)

window = 20
dim = 5
y, eigs = stride_eig(sig, window, dim, return_eigs=True)


# sub = 3
# figure(figsize = (15,1), dpi = 80)
# for i,e in enumerate(eigs):
#     if i % sub == 0 and i*sub+dim <N:
#         plt.plot(x[i*sub:i*sub+dim],e)

# plt.show()

plt.plot(x, sig, label='Signal');
[plt.plot(x, c, '--', label=f'C{i}') for i,c in enumerate(basis)]
# plt.plot(x[:N-window], y, label='spikes');
plt.plot(x, normalize(y), label='spikes normalized');
plt.legend()
plt.show()

# plot some eigenvectors
num = 20
figure(figsize = (5, 10), dpi = 80)
plt.plot(np.mean(eigs, axis=0)-2)
[plt.plot(e+i/2) for i,e in enumerate(eigs[::num]) ]
plt.xlim([-dim,dim*2])
plt.title(f"Some Eigenvectors")
plt.show()



In [ ]:
resPH, stimPH = read_lmc('PHOTO');
res3, stim3 = read_lmc('LMC-BG3');

In [ ]:

sig = np.mean(stimPH, axis=0)
res = np.mean(resPH, axis=0)
# i = 10
# sig = stimPH[i]
# res = resPH[i]

down = 10
sig = normalize(sig[::down])
res = normalize(res[::down])

plt.plot(sig); plt.title("Stimulus"); plt.show()

window = 200
dim = 50
time_eig, eigs = stride_eig(sig, window, dim, shift=1, stride=1, return_eigs=True)
full_eig, evec = max_eig_project(sig, dim, shift=1)

figure(figsize = (8, 8), dpi = 80)
plt.plot(evec)
plt.title(f"Linear Fit Eigenvectors")
plt.show()

time_eig = normalize(time_eig)
full_eig = normalize(full_eig)

lags, corr = cross_corr(res, sig)
plt.plot(lags, corr, label='CC Response-Sig');

lags, corr = cross_corr(res, time_eig)
plt.plot(lags, corr, label='CC Response-Time_Eig');
max_corr_index = lags[np.argmax(corr)]

lags, corr = cross_corr(res, full_eig)
plt.plot(lags, corr, label='CC Response-Full_Eig');
max_corr_index_full = lags[np.argmax(corr)]

plt.title("Cross Correlations")
plt.xticks(np.arange(-1000,1000,100)); plt.legend(); plt.show();

norm_res = res[max_corr_index:]
plt.plot(norm_res, label="Neuron Response")
plt.plot(time_eig, label="Eigentime Projection")
plt.legend(); plt.show();

norm_res = res[max_corr_index_full:]
plt.plot(norm_res, label="Neuron Response")
plt.plot(full_eig, label="Full_Eig Projection")
plt.legend(); plt.show();





In [ ]:

figure(figsize = (40, 5), dpi = 80)
x = np.linspace(0,1,N)
avg_e = np.zeros(N-window+dim)
sub = 8
for i,e in enumerate(eigs):
    if i % sub == 0 and i*sub+dim <N:
        plt.plot(x[i*sub:i*sub+dim],e)
    avg_e[i:i+dim] += e
# plt.plot(x[:len(avg_e)], avg_e);
plt.plot(x, res)
plt.show()

num = 15
figure(figsize = (5, 15), dpi = 80)
plt.plot(np.mean(eigs, axis=0)-2)
[plt.plot(e+i/2) for i,e in enumerate(eigs[::num]) ]
plt.xlim([-len(e),len(e)*2])
plt.title(f"Some Eigenvectors")
plt.show()

In [ ]:
from projectors import *

mse = lambda A,B: np.square(np.subtract(A, B)).mean()

down = 2

sig = normalize(np.mean(stimPH[:, ::down], axis=0))
res = normalize(np.mean(resPH[:, ::down], axis=0))

train_test_ratio = 0.8
split = int(len(sig) * train_test_ratio)
X_train, X_test = sig[:split], sig[split:]
Y_train, Y_test = res[:split], res[split:]

dim = 10000

P_train, theta = linear_model(X_train, Y_train, dim, reg=10)

figure(figsize = (8, 8), dpi = 80)
plt.plot(theta)
plt.title(f"Linear Fit Eigenvectors")
plt.show()



plt.plot(Y_train, label="True");
plt.plot(P_train, label="Pred");
plt.title(f"TRAIN Linear Fit Sig"); plt.legend()
plt.show()


pad_test_sig = np.concatenate([np.zeros(dim - 1), X_test])
test_Han = build_hankel(pad_test_sig, dim).T
P_test = test_Han @ theta 

plt.plot(Y_test, label="True")
plt.plot(P_test, label="Pred")
plt.title(f"TEST Linear Fit Sig"); plt.legend()
plt.show()

train_err = mse(Y_train, P_train)
test_err = mse(Y_test, P_test)
print(f'Train Error: {train_err}')
print(f'Test Error: {test_err}')
